In [1]:
# setting environment
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# supress warnings
import warnings
warnings.filterwarnings("ignore")

### Description
This is the data set from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult), extraction was done by Barry Becker from the 1994 Census database.

### Goal
Prediction task is to determine whether a person makes over 50K a year. 

In [2]:
# read in data
adult_raw = pd.read_csv('data/adult.data', header=None)

In [3]:
# check the shape of data
adult_raw.shape

(32561, 15)

In [4]:
# show the first few rows
adult_raw.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


We can see that the variable names are not informative, we will extract the column names from the original web page

In [5]:
# rename the column
col = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
      'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
adult_raw.columns = col

In [6]:
# check the result
adult_raw.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
# check the number of unique value in each column
adult_raw.nunique()

age                  73
workclass             9
fnlwgt            21648
education            16
education_num        16
marital_status        7
occupation           15
relationship          6
race                  5
sex                   2
capital_gain        119
capital_loss         92
hours_per_week       94
native_country       42
income                2
dtype: int64

In [8]:
# check the data type
adult_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [9]:
# examine each object column
for i in adult_raw.select_dtypes(include=['object']).columns:
    print(adult_raw[i].value_counts())

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64
 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital_status, dtype: int64
 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial

We can see that there are missing values in `workclass`, `occupation` and `native_country`, it is showing '?' instead of the null value, we will convert the '?' to null

In [10]:
# strip the leading and trailing space in each column
adult_raw = adult_raw.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# replace '?' with np.nan
adult_raw = adult_raw.replace('?', np.nan)

In [11]:
# check the number of missing values in each column
adult_raw.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

In [12]:
# check the target column
adult_raw['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

In [13]:
# transform the income column into binary, assign 1 if >50K, 0 otherwise

# strip the starting and trailing space
adult_raw['income'] = adult_raw['income'].str.strip()

# map the result
adult_raw['income'] = adult_raw['income'].map({'<=50K':0, '>50K': 1})

In [14]:
# check the result
adult_raw['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [15]:
adult_raw.shape

(32561, 15)

In [16]:
adult_raw.dropna().shape

(30162, 15)

Since there is relatively small portion of missing values compared to our dataset, as the first approach, we will drop the missing values row directly

In [17]:
adult_clean = adult_raw.dropna()

In [18]:
# identify the numeric column for further use
num_col = adult_clean.select_dtypes(include=['number']).drop('income', axis=1).columns

In [19]:
num_col

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [100]:
# convert the categorical variable to dummies
adult_clean = pd.get_dummies(adult_clean)

In [101]:
# split the data into traing and testing set, stratify with income so the ratio between 1 and 0 are the same in 
# both train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(adult_clean.drop(['income'], axis=1), adult_clean['income'],
                                                   test_size=0.3, random_state=1009, stratify=adult_clean['income'])

Since there are some missing values in both traing and testing set as expected, we will deal with them first

In [102]:
# import StandardScaler to normalize the column
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# extract the numerical column
X_train[num_col] = scale.fit_transform(X_train[num_col])
X_test[num_col] = scale.transform(X_test[num_col])

In [103]:
# check the result
X_train.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
11990,1.720246,0.076117,-0.440755,-0.146977,-0.218844,-0.081840,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
29558,1.949223,0.267469,1.136275,3.719404,-0.218844,1.171884,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
20635,0.575364,-0.821204,1.136275,-0.146977,-0.218844,-0.081840,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
19614,0.575364,0.041928,-0.440755,-0.146977,-0.218844,-0.081840,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
8832,-0.187891,-1.334824,1.530532,-0.146977,-0.218844,-0.081840,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Now we are ready to fit the model, we will try gradient boosting and random forest first before we train our neural network model

In [104]:
# import a best model gridsearch to tune the hyperparameter
from sklearn.model_selection import GridSearchCV
def get_best_model_accuracy(model, params, X, y):
    grid = GridSearchCV(model, params, error_score=0., cv=5)
    grid.fit(X, y)
    # our classical metric for performance
    print("Best Accuracy: {}".format(grid.best_score_))
    # the best parameters that caused the best accuracy
    print("Best Parameters: {}".format(grid.best_params_))
    # the average time it took a model to fit to the data (in seconds)
    print("Average Time to Fit (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    # the average time it took a model to predict out of sample data (in seconds)
    # this metric gives us insight into how this model will perform in real-time analysis
    print("Average Time to Score (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))
    #return grid.best_params_
   
    

In [105]:
# base score to beat
adult_raw['income'].value_counts(normalize=True)

0    0.75919
1    0.24081
Name: income, dtype: float64

We can see from above that our base score to beat is 75.92%, we will try random forest and gradient boosting first

### Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_params = {'n_estimators': [50, 100, 300, 500],
             'criterion': ['gini', 'entropy'],
             'max_depth': [3, 5, 7, 9]} 
get_best_model_accuracy(rf, rf_params, X_train, y_train)

Best Accuracy: 0.8510396438213423
Best Parameters: {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 300}
Average Time to Fit (s): 1.849
Average Time to Score (s): 0.1


### Gradient Boosting

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
gbr = GradientBoostingClassifier()
gbr_params = {'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
              'n_estimators': [50, 100, 500]}
get_best_model_accuracy(gbr, gbr_params, X_train, y_train)

Best Accuracy: 0.8659593615308104
Best Parameters: {'learning_rate': 0.1, 'n_estimators': 500}
Average Time to Fit (s): 9.471
Average Time to Score (s): 0.018


### MLP Classifier

In [28]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp_params = {'hidden_layer_sizes': [(100, 50), (100, 50, 50), (100, 50, 50 ,30)],
              'learning_rate_init': [0.001, 0.005, 0.01]}
get_best_model_accuracy(mlp, mlp_params, X_train, y_train)

Best Accuracy: 0.8205371098375408
Best Parameters: {'hidden_layer_sizes': (100, 50, 50), 'learning_rate_init': 0.01}
Average Time to Fit (s): 34.509
Average Time to Score (s): 0.019


From the hyper parameter tuning result, we can see that MLP classifier actually performed the worst and it took a lot of time to train as well. We will test the test set for each model with best hyperparameters

In [29]:
rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=9)
rf.fit(X_train, y_train)
print('Random forest performance: {}'.format(rf.score(X_test, y_test)))

Random forest performance: 0.853243452315173


In [30]:
gbr = GradientBoostingClassifier(learning_rate=0.1, n_estimators=500)
gbr.fit(X_train, y_train)
print('Gradient boosting performance: {}'.format(gbr.score(X_test, y_test)))

Gradient boosting performance: 0.8735771908498177


In [31]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50, 50), learning_rate_init=0.01)
mlp.fit(X_train, y_train)
print('Multiple layer neural network performance: {}'.format(mlp.score(X_test, y_test)))

Multiple layer neural network performance: 0.834567355508896


### Second approach, imputing without dropping examples with missing values
We missed around 2500 training examples from dropping the rows with missing values, this time we will use the imputing method to fill up those values to see whether it increase the performance

In [74]:
adult_raw.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

In [106]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy='most_frequent')

col = ['workclass', 'occupation', 'native_country']
adult_impute = adult_raw.copy()
adult_impute[col] = impute.fit_transform(adult_impute[col])

In [108]:
# convert the categorical variable to dummies
adult_impute = pd.get_dummies(adult_impute)

In [109]:
# split the data into traing and testing set, stratify with income so the ratio between 1 and 0 are the same in 
# both train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(adult_impute.drop(['income'], axis=1), adult_impute['income'],
                                                   test_size=0.3, random_state=1009, stratify=adult_impute['income'])

### Random Forest

In [110]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_params = {'n_estimators': [50, 100, 300, 500],
             'criterion': ['gini', 'entropy'],
             'max_depth': [3, 5, 7, 9]} 
get_best_model_accuracy(rf, rf_params, X_train, y_train)

Best Accuracy: 0.8565724815724816
Best Parameters: {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 500}
Average Time to Fit (s): 1.971
Average Time to Score (s): 0.107


### Gradient Boosting

In [111]:
from sklearn.ensemble import GradientBoostingClassifier
gbr = GradientBoostingClassifier()
gbr_params = {'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
              'n_estimators': [50, 100, 500]}
get_best_model_accuracy(gbr, gbr_params, X_train, y_train)

Best Accuracy: 0.8716654966654966
Best Parameters: {'learning_rate': 0.1, 'n_estimators': 500}
Average Time to Fit (s): 9.876
Average Time to Score (s): 0.021


### MLP Classifier

In [112]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp_params = {'hidden_layer_sizes': [(100, 50), (100, 50, 50), (100, 50, 50 ,30)],
              'learning_rate_init': [0.001, 0.005, 0.01]}
get_best_model_accuracy(mlp, mlp_params, X_train, y_train)

Best Accuracy: 0.8045366795366795
Best Parameters: {'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.005}
Average Time to Fit (s): 10.844
Average Time to Score (s): 0.021


In [113]:
rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=9)
rf.fit(X_train, y_train)
print('Random forest performance: {}'.format(rf.score(X_test, y_test)))

Random forest performance: 0.8559729757395844


In [114]:
gbr = GradientBoostingClassifier(learning_rate=0.1, n_estimators=500)
gbr.fit(X_train, y_train)
print('Gradient boosting performance: {}'.format(gbr.score(X_test, y_test)))

Gradient boosting performance: 0.8733749616132664


In [115]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), learning_rate_init=0.005)
mlp.fit(X_train, y_train)
print('Multiple layer neural network performance: {}'.format(mlp.score(X_test, y_test)))

Multiple layer neural network performance: 0.7985464223564336
